In [1]:
import torch
import torch.nn as nn
from utils import Data, plot_losses, calculate_probability, flip_from_probability, save_data, load_data
from model import StockGPT2Model
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [2]:
file_path = "/home/summer_20/Divyam/StockGPT/model/"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

batch_size = 4
# data.completed_tickers.append("LDO-USD")
data = load_data(file_path)
model = StockGPT2Model(num_features = data.num_features, num_tickers = data.num_tickers).to(device)
model.load_state_dict(torch.load("/home/summer_20/Divyam/StockGPT/model/best_model_weights.pth"))
data_loader = DataLoader(data, batch_size=batch_size, shuffle=False)

Device: cuda


Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized because the shapes did not match:
- h.0.attn.c_attn.bias: found shape torch.Size([2304]) in the checkpoint and torch.Size([756]) in the model instantiated
- h.0.attn.c_attn.weight: found shape torch.Size([768, 2304]) in the checkpoint and torch.Size([252, 756]) in the model instantiated
- h.0.attn.c_proj.bias: found shape torch.Size([768]) in the checkpoint and torch.Size([252]) in the model instantiated
- h.0.attn.c_proj.weight: found shape torch.Size([768, 768]) in the checkpoint and torch.Size([252, 252]) in the model instantiated
- h.0.ln_1.bias: found shape torch.Size([768]) in the checkpoint and torch.Size([252]) in the model instantiated
- h.0.ln_1.weight: found shape torch.Size([768]) in the checkpoint and torch.Size([252]) in the model instantiated
- h.0.ln_2.bias: found shape torch.Size([768]) in the checkpoint and torch.Size([252]) in the model instantiated
- h.0.ln_2.w

In [3]:
criterion = nn.MSELoss()
with torch.no_grad():
    model.eval()
    data.mode = "val"
    all_outputs = []
    all_targets = []
    val_loss = 0
    for batch_idx, (src, sos) in enumerate(data_loader):
        src, sos = src.to(device), sos.to(device)
        temp_src = src[:,:data.len_input]
        # Forward pass
        output, past_key_values = model(src = temp_src, sos = sos)
        temp_src = output[:,-1,:].unsqueeze(1)
        for i in range(data.len_input, data.len_input + data.len_output - 1):
            temp_output, past_key_values = model(src = temp_src, past_key_values=past_key_values)
            temp_src = temp_output
            output = torch.cat((output,temp_output), dim = 1)
        
        # output = model(batch_input, batch_target[:,:-1], sos)
        # print(output)
      # Reshape output and batch_target
        output = output.reshape(-1, data.num_features)  # [batch_size * seq_len, 3]
        src = src.view(-1, data.num_features)  # [batch_size * seq_len, 3]

        all_outputs.append(output.to(device))
        all_targets.append(src)
        loss = criterion(output.to(device), src)
        val_loss += loss.item()
    val_loss = val_loss / len(data_loader)

# Concatenate all batches
all_outputs = torch.cat(all_outputs, dim=0)
all_targets = torch.cat(all_targets, dim=0)
print(val_loss)

0.03421285442334223


In [4]:
fig = plt.figure(figsize=(8, 6))
for index in range(100):
    fig.clear()
    plt.plot(all_targets[index*800:(index+1)*800, 0].to("cpu"), label="Target", color="orange")
    plt.plot(all_outputs[index*800:(index+1)*800, 0].to("cpu"), label="Output", color="blue")

    plt.xlabel("Time Step")
    plt.ylabel(f"Feature {0+1}")
    plt.title(f"Feature {0+1} - Target vs. Output")
    plt.legend()
    plt.grid(True)
    plt.waitforbuttonpress()

In [6]:
from sklearn.metrics import r2_score

# Calculate R2 score for each output dimension
r2_scores = r2_score(all_targets.cpu(), all_outputs.cpu(), multioutput='raw_values')

print("R2 Scores for each output dimension:", r2_scores)

R2 Scores for each output dimension: [0.70136543 0.68302082 0.68703538 0.69523595 0.51779762]
